In [3]:
#######
## if you are unfamiliar with jupyter notebook, take a look at this tutorial
## https://www.dataquest.io/blog/jupyter-notebook-tutorial/
##
## make sure all dependencies are installed
## pip install --upgrade numpy open3d
#######
import open3d as o3d # http://www.open3d.org/
import numpy as np
import time

print(f"open3d :{o3d.__version__}")

open3d :0.11.1


In [4]:
def setView(ctr,camera_pos=(1, 1, 1), lookat=(0, 0, 0), up=(0, 0, 1)):
    """
    set the view given a view control handel ctr
    """
    ctr.set_constant_z_far(100) # camera z far clip plane
    ctr.set_constant_z_near(0.01)# camera z near clip plane
    
def customVisualization(geomtry_list):
    """
    helper function to create a visualization given a list of o3d geometry
    """
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    for g in geomtry_list:
        vis.add_geometry(g)
    ctr = vis.get_view_control()
    setView(ctr)
    vis.run()
    vis.destroy_window() # close the window when finished

In [5]:
def createGrid(bounds=[[-1, -1, -1], [1, 1, 1]], dr=0.1):
    """
    retrun a grid of points shaped (n,3) given bounds and discritization radius
    the bounds are updated and also returned
    input:
        bounds: [(x_low,y_low,z_low),(x_high,y_high,z_high)]
        dr: discretization radius of the grid
    output:
        xyz_grid: a grid of points numpy array of (n,3)
        bounds: updated bounds
        nx,ny,nz: number of points in x,y,z direction
    """
    # round to integer, type is still float
    bounds = bounds/dr
    bounds = np.stack((np.floor(bounds[0]), np.ceil(bounds[1])))*dr
#     print("bounds=\n", bounds)
    # number of points in x,y,z direction:(nx,ny,nz)
    nx, ny, nz = np.ceil((bounds[1]-bounds[0])/dr).astype(int)
    x = np.linspace(bounds[0, 0], bounds[0, 0]+(nx-1)*dr, num=nx)
    y = np.linspace(bounds[0, 1], bounds[0, 1]+(ny-1)*dr, num=ny)
    z = np.linspace(bounds[0, 2], bounds[0, 2]+(nz-1)*dr, num=nz)
    # a flattened grid of xyzs of the vertices
    xyz_grid = np.stack(np.meshgrid(x, y, z), axis=-1).reshape((-1, 3))
    return xyz_grid, bounds, (nx, ny, nz)

def createPlane(r=0.5, dr=0.1):
    """
    return a plane located at (0,0,0),and with plane normal = (0,0,1)
    r: radius of the plane
    dr:discretization radius of the grid
    """
    bounds = np.array([[-r, -r, 0],[r, r, 0]])/dr
    bounds = np.stack((np.floor(bounds[0]), np.ceil(bounds[1])))*dr
    nx, ny, nz = np.ceil((bounds[1]-bounds[0])/dr).astype(int)
#     print(nx,ny)
    xyz = np.reshape([[[[i, j, 0], [i+1, j, 0], [i, j+1, 0],
                       [i, j+1, 0], [i+1, j, 0], [i+1, j+1, 0]] for i in range(nx-1)] for j in range(ny-1)], (-1, 3))
    xyz = (xyz - ((nx-1)/2,(ny-1)/2,0))*dr
#     xyz, bounds, (nx, ny, nz) = create_grid(bounds, dr)
#     print(nx, ny, nz)
    triangles = np.arange(xyz.shape[0]).reshape((-1,3))
    plane = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(
        xyz), o3d.utility.Vector3iVector(triangles))
    # assign checkerboard color pattern
    c0 = (0.729, 0.78, 0.655) # first color
    c1 = (0.533, 0.62, 0.506) # second color
    colors = np.reshape([[np.tile(c0 if (i+j)%2 else c1,(6,1)) for i in range(nx-1)] for j in range(ny-1)],(-1,3))
    plane.vertex_colors = o3d.utility.Vector3dVector(colors)
    plane.compute_triangle_normals()
    return plane

# create a plane
plane = createPlane()
# create a coordinate frame
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])

# o3d.visualization.draw_geometries([plane, coord_frame],mesh_show_wireframe=True)
customVisualization([plane, coord_frame])

In [4]:
##################### create the points #####################################
bounds = np.array([[-1,-1,-1],[1,1,1]])*0.1 # [lower_bound,upper_bound]
radius_grid = 0.2 # discretization radius of the grid
mass_pos, bounds,_ = createGrid(bounds = bounds, dr = radius_grid) # create a grid of masses (xyzs)
print('mass_pos', mass_pos)
mass_pos = mass_pos + (0,0,-mass_pos[:,-1].min()) # move the vertices to z>=0
print('mass_pos', mass_pos)
# setup the point cloud data
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(mass_pos)

######################## create the edges ##################################
# KDTree for nearest neighbor search
pcd_tree = o3d.geometry.KDTreeFlann(pcd)
radius_knn = radius_grid*np.sqrt(3)*1.01
max_nn = 28
neighbors = [np.asarray(pcd_tree.search_hybrid_vector_3d(point, radius_knn,max_nn = max_nn)[1]) for point in mass_pos]

def GetEdges(neighbor):
    candidate = neighbor[1:]
    self = neighbor[0]
    candidate = candidate[candidate<self] # to remove redundency
    edges = np.empty((candidate.size,2),dtype=np.int32)
    edges[:,0]=self
    edges[:,1]=candidate
    return edges

edges = np.vstack([GetEdges(neighbor[:max_nn]) for neighbor in neighbors])

# create the line set data
lsd = o3d.geometry.LineSet()
lsd.points = o3d.utility.Vector3dVector(mass_pos) # (n by 3) arry of points xyzs
lsd.lines = o3d.utility.Vector2iVector(edges) # (n by 2) indices of the edges

print(mass_pos.shape)
print(edges.shape)
print(edges)

mass_pos [[-0.2 -0.2 -0.2]
 [-0.2 -0.2  0. ]
 [ 0.  -0.2 -0.2]
 [ 0.  -0.2  0. ]
 [-0.2  0.  -0.2]
 [-0.2  0.   0. ]
 [ 0.   0.  -0.2]
 [ 0.   0.   0. ]]
mass_pos [[-0.2 -0.2  0. ]
 [-0.2 -0.2  0.2]
 [ 0.  -0.2  0. ]
 [ 0.  -0.2  0.2]
 [-0.2  0.   0. ]
 [-0.2  0.   0.2]
 [ 0.   0.   0. ]
 [ 0.   0.   0.2]]
(8, 3)
(28, 2)
[[1 0]
 [2 0]
 [2 1]
 [3 1]
 [3 2]
 [3 0]
 [4 0]
 [4 1]
 [4 2]
 [4 3]
 [5 1]
 [5 4]
 [5 0]
 [5 3]
 [5 2]
 [6 2]
 [6 4]
 [6 0]
 [6 3]
 [6 5]
 [6 1]
 [7 3]
 [7 5]
 [7 6]
 [7 1]
 [7 2]
 [7 4]
 [7 0]]


In [5]:
customVisualization([pcd,lsd,plane, coord_frame])

NameError: name 'pcd' is not defined

In [5]:
def NonBlockingVisualization():
    """
    example for the non-blocking visualization
    """    
    ended=False
    def signalEnd(vis):
        nonlocal ended
        ended =True
        
    vis = o3d.visualization.VisualizerWithKeyCallback()
    
    # ref:https://www.glfw.org/docs/3.3/group__keys.html
    # press key Q or ESC to close the window
    vis.register_key_callback(81, signalEnd)# key Q 
    vis.register_key_callback(256, signalEnd)# key escape
    vis.create_window()

    # add geometry
    vis.add_geometry(lsd)
    vis.add_geometry(plane)
    vis.add_geometry(pcd)
    
    # view control
    ctr = vis.get_view_control()
    setView(ctr)
    
    mass_pos_t = np.copy(mass_pos)
    
    t_prev = time.time() # previous time
    while (not ended):
        t = time.time()
        
        # change something here
        # e.g. change mass_pos_t
        mass_pos_t= mass_pos+np.sin(t*10)*0.01

        if t - t_prev>1./60.:# update renderer
            t_prev = t
            lsd.points = o3d.utility.Vector3dVector(mass_pos_t)
            vis.update_geometry(lsd)
            pcd.points = o3d.utility.Vector3dVector(mass_pos_t)
            vis.update_geometry(pcd)
            vis.poll_events()
            vis.update_renderer()
            
    vis.destroy_window() # close the window when finished
    
NonBlockingVisualization()

In [68]:
# Step 1 DS for masses and springs
class Mass(object):
    """docstring for Mass"""
    
    def __init__(self, index, m, p, v=np.zeros((3, 1)), a=np.zeros((3, 1)), F=np.zeros((3, 1))):
        super(Mass, self).__init__()
        self.m = m # in kg
        self.p = p
        self.v = v
        self.a = a
        self.F = F
        self.index = index
        self.ground_k = 10000

    def update_pos(self, dt):
        # print('self.F', self.F)
        # Ground Force
        
        if self.p[2] < 0:
            self.F[2] = self.F[2] + self.ground_k * (-self.p[2])
            
        # gravity

        self.a = self.F / self.m
        self.a = self.a + g
        # print('self.a', self.a)
        # print('self.a * dt', self.a * dt)
        # print('self.v', self.v)
        # print('self.v + self.a * dt', self.v + self.a * dt)
        self.v = self.v + self.a * dt
        self.v = self.v * 0.999
        # print('self.v', self.v)
        self.p = self.p + self.v * dt

class Spring(object):
    """docstring for Spring"""
    def __init__(self, k, l_0, m1_index, m2_index):
        super(Spring, self).__init__()
        self.k = k # spring constant
        self.l_0 = l_0 # rest length in meter
        self.m1_index = m1_index # index od the first mass it connects to
        self.m2_index = m2_index


In [74]:
def distance(m1, m2):
    return np.linalg.norm(m1.p-m2.p)


def direction(m1, m2):
    # m2 to m1 
    vector = m2.p - m1.p
    vector /= np.linalg.norm(vector)
    return vector


##################### create the points #####################################
bounds = np.array([[-1,-1,-1],[1,1,1]])*0.1 # [lower_bound,upper_bound]
radius_grid = 0.2 # discretization radius of the grid
mass_pos, bounds,_ = createGrid(bounds = bounds, dr = radius_grid) # create a grid of masses (xyzs)
print('mass_pos', mass_pos)
# mass_pos = mass_pos + (0,0,-mass_pos[:,-1].min()) # move the vertices to z>=0
mass_pos = mass_pos + (0,0,0.2) # move the vertices to z>=0
print('mass_pos', mass_pos)
# setup the point cloud data
mass_ls = []
for i, pos in enumerate(mass_pos):
    mass = Mass(i, m=0.1, p=(pos.reshape(3, 1)))
    mass_ls.append(mass)
    
spring_ls = []
for i in range(len(mass_ls)):
    for j in range(i+1, len(mass_ls)):
        k = 10000
        l_0 = distance(mass_ls[i], mass_ls[j])

        spring = Spring(k, l_0, i, j)
        spring_ls.append(spring)




# Step 3 Global variable
g = np.array([0, 0, -1]).reshape((3, 1))
dt = 0.0001
t = 0


print()

mass_pos [[-0.2 -0.2 -0.2]
 [-0.2 -0.2  0. ]
 [ 0.  -0.2 -0.2]
 [ 0.  -0.2  0. ]
 [-0.2  0.  -0.2]
 [-0.2  0.   0. ]
 [ 0.   0.  -0.2]
 [ 0.   0.   0. ]]
mass_pos [[-0.2 -0.2  0. ]
 [-0.2 -0.2  0.2]
 [ 0.  -0.2  0. ]
 [ 0.  -0.2  0.2]
 [-0.2  0.   0. ]
 [-0.2  0.   0.2]
 [ 0.   0.   0. ]
 [ 0.   0.   0.2]]



In [75]:
mass_pos_ls = []
for mass in mass_ls:
    mass_pos_ls.append(mass.p)
mass_pos = np.array(mass_pos_ls).reshape((8, 3))
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(mass_pos)

######################## create the edges ##################################
# KDTree for nearest neighbor search
pcd_tree = o3d.geometry.KDTreeFlann(pcd)
radius_knn = radius_grid*np.sqrt(3)*1.01
max_nn = 28
neighbors = [np.asarray(pcd_tree.search_hybrid_vector_3d(point, radius_knn,max_nn = max_nn)[1]) for point in mass_pos]

def GetEdges(neighbor):
    candidate = neighbor[1:]
    self = neighbor[0]
    candidate = candidate[candidate<self] # to remove redundency
    edges = np.empty((candidate.size,2),dtype=np.int32)
    edges[:,0]=self
    edges[:,1]=candidate
    return edges

edges = np.vstack([GetEdges(neighbor[:max_nn]) for neighbor in neighbors])

# create the line set data
lsd = o3d.geometry.LineSet()
lsd.points = o3d.utility.Vector3dVector(mass_pos) # (n by 3) arry of points xyzs
lsd.lines = o3d.utility.Vector2iVector(edges) # (n by 2) indices of the edges

In [76]:

def DropCubeVisualization():
    """
    example for the non-blocking visualization
    """    
    ended=False
    def signalEnd(vis):
        nonlocal ended
        ended =True
        
    vis = o3d.visualization.VisualizerWithKeyCallback()
    
    # ref:https://www.glfw.org/docs/3.3/group__keys.html
    # press key Q or ESC to close the window
    vis.register_key_callback(81, signalEnd)# key Q 
    vis.register_key_callback(256, signalEnd)# key escape
    vis.create_window()

    # add geometry
    vis.add_geometry(lsd)
    vis.add_geometry(plane)
    vis.add_geometry(pcd)
    
    # view control
    ctr = vis.get_view_control()
    setView(ctr)
    
    mass_pos_t = np.copy(mass_pos)
    
#     t_prev = time.time() # previous time
#     while (not ended):
#         t = time.time()
        
#         # change something here
#         # e.g. change mass_pos_t
#         mass_pos_t= mass_pos+np.sin(t*10)*0.01

#         if t - t_prev>1./60.:# update renderer
#             t_prev = t
#             lsd.points = o3d.utility.Vector3dVector(mass_pos_t)
#             vis.update_geometry(lsd)
#             pcd.points = o3d.utility.Vector3dVector(mass_pos_t)
#             vis.update_geometry(pcd)
#             vis.poll_events()
#             vis.update_renderer()
    t = 0
    t_prev = 0
    while (not ended):
        t += dt
        for spring in spring_ls:
            m1 = mass_ls[spring.m1_index]
            m2 = mass_ls[spring.m2_index]
            l_0 = spring.l_0
            l = distance(m1, m2)
            F = spring.k * (l - l_0) # check the direction
            F = direction(m1, m2) * F
            F = F.reshape(F.shape[0], -1)
            mass_ls[spring.m1_index].F, mass_ls[spring.m2_index].F = F, -F

        for i in range(len(mass_ls)):
            mass_ls[i].update_pos(dt)

        if t - t_prev > 1./60.:# update renderer
            t_prev = t
            mass_pos_ls = []
            for mass in mass_ls:
                mass_pos_ls.append(mass.p)
            mass_pos_t = np.array(mass_pos_ls).reshape((8, 3))
            lsd.points = o3d.utility.Vector3dVector(mass_pos_t)
            vis.update_geometry(lsd)
            pcd.points = o3d.utility.Vector3dVector(mass_pos_t)
            vis.update_geometry(pcd)
            vis.poll_events()
            vis.update_renderer()

    
    vis.destroy_window() # close the window when finished
    
DropCubeVisualization()

In [72]:
t = 0
t_prev = 0
mass_pos_t = np.copy(mass_pos)
print('mass_pos_t ', mass_pos_t[0])
while t < 100:
    t += dt
    for spring in spring_ls:
        m1 = mass_ls[spring.m1_index]
        m2 = mass_ls[spring.m2_index]
        l_0 = spring.l_0
        l = distance(m1, m2)
        F = spring.k * (l - l_0) # check the direction
        F = direction(m1, m2) * F
        F = F.reshape(F.shape[0], -1)
        mass_ls[spring.m1_index].F, mass_ls[spring.m2_index].F = F, -F

    for i in range(len(mass_ls)):
        mass_ls[i].update_pos(dt)

    if t - t_prev> 1./6.:# update renderer
        t_prev = t
        mass_pos_ls = []
        for mass in mass_ls:
            mass_pos_ls.append(mass.p)
        mass_pos_t = np.array(mass_pos_ls).reshape((8, 3))
#         print('mass_pos_t ', mass_pos_t[0])

mass_pos_t  [-0.2 -0.2  0. ]


KeyboardInterrupt: 